In [1]:
import json
import os
import sys

app_env = os.getenv("APP_ENV", "local")
app_files_path = os.getenv("APP_FILES_PATH")

# %pip install $app_files_path

sys.path.append((app_files_path if app_env != "local" else os.path.abspath(os.path.join(".."))))

In [2]:
from invaas.robinhood.robinhood_task import RobinhoodTask

task = RobinhoodTask(env=app_env)
task.setup_api()

2024-10-16 13:50:31 WARNING No DBUtils defined in the runtime
2024-10-16 13:50:31 INFO Initializing task for local environment
2024-10-16 13:50:31 INFO Logging in to Robinhood
2024-10-16 13:50:31 INFO Available cash: $1397.84


In [3]:
# task.robinhood_api.load_account_profile()

In [4]:
rsi = task.get_rsi()
(
    current_fear_greed_index_timestamp,
    current_fear_greed_index,
    previous_max_fear_greed_index,
    previous_min_fear_greed_index,
) = task.get_fear_greed_index_data()

2024-10-16 13:50:32 INFO Current SPY RSI: 61
2024-10-16 13:50:32 INFO Current fear greed index timestamp: 2024-10-16 00:00:00
2024-10-16 13:50:32 INFO Current fear greed index: 68
2024-10-16 13:50:32 INFO Previous max fear greed index: 75
2024-10-16 13:50:32 INFO Previous min fear greed index: 67


In [5]:
import numpy as np
import pandas as pd

from datetime import date, timedelta
from multiprocessing.pool import ThreadPool


def get_friday_n_days_from_today(n):
    next_day = date.today() + timedelta(days=n)
    return str(next_day + timedelta((4 - next_day.weekday()) % 7))


expiration_date = get_friday_n_days_from_today(task.periods)
print(f"Expiration date: {expiration_date}")


def find_options_by_specific_profitability_threaded(
    inputSymbols,
    expirationDate=None,
    strikePrice=None,
    optionType=None,
    typeProfit="chance_of_profit_short",
    profitFloor=0.0,
    profitCeiling=1.0,
    info=None,
):
    symbol = inputSymbols
    tempData = task.robinhood_api.find_tradable_options(symbol, expirationDate, strikePrice, optionType, info=None)
    tempData = [x for x in tempData if x.get("expiration_date") == expirationDate]

    def get_option_data(option):
        market_data = task.robinhood_api.get_option_market_data_by_id(option["id"])

        if len(market_data):
            option.update(market_data[0])

            try:
                floatValue = float(option[typeProfit])
                if floatValue >= profitFloor and floatValue <= profitCeiling:
                    return option
            except Exception as e:
                print(e)
                pass

    pool = ThreadPool(processes=12)
    data = [x for x in pool.map(get_option_data, tempData) if x]

    return task.robinhood_api.filter_data(data, info)


print("Getting options...")

# options_data = task.robinhood_api.find_options_by_specific_profitability(
options_data = find_options_by_specific_profitability_threaded(
    inputSymbols=task.symbol, expirationDate=expiration_date, profitFloor=0.95
)

df_options = pd.DataFrame(data=options_data)

# df_options

Expiration date: 2024-11-08
Getting options...


Found Additional pages.
Loading page 2 ...
Loading page 3 ...
2024-10-16 13:50:36 WARNING Connection pool is full, discarding connection: api.robinhood.com. Connection pool size: 10
2024-10-16 13:50:36 WARNING Connection pool is full, discarding connection: api.robinhood.com. Connection pool size: 10


In [6]:
min_amount = 10
min_buy_price = 0.2
max_buy_price = task.available_cash / 10 / 100
# max_buy_price = 200
round_decimals = 2

print(f"Min buy price: ${min_buy_price:.2f}")
print(f"Max buy price: ${max_buy_price:.2f}")

numeric_columns = [
    "chance_of_profit_short",
    "strike_price",
    "ask_size",
    "bid_size",
    "last_trade_size",
    # "open_interest",
    # "volume",
    # "mark_price",
    "ask_price",
    "bid_price",
    "last_trade_price",
    # "high_fill_rate_buy_price",
    # "high_fill_rate_sell_price",
]

for column in numeric_columns:
    df_options[column] = np.round(pd.to_numeric(df_options[column]), decimals=round_decimals)

# df_options["high_fill_rate_price_diff"] = np.round(
#     df_options.high_fill_rate_sell_price - df_options.high_fill_rate_buy_price, decimals=round_decimals
# )

df_options_filtered = df_options.loc[
    (df_options.tradability == "tradable")
    & (df_options.rhs_tradability == "tradable")
    & (df_options.state == "active")
    & (df_options.ask_size > min_amount)
    & (df_options.bid_size > min_amount)
    # & (df_options.last_trade_size > min_amount)
    & (df_options.open_interest > min_amount)
    & (df_options.volume > min_amount)
    & (df_options.ask_price > min_buy_price)
    & (df_options.ask_price < max_buy_price)
    # & (df_options.high_fill_rate_buy_price < df_options.high_fill_rate_sell_price)
]

# df_options_filtered = df_options_filtered.sort_values(
#     by=["high_fill_rate_price_diff", "chance_of_profit_short"],
#     # by=["chance_of_profit_short", "high_fill_rate_price_diff"],
#     ascending=[False, False],
# )[
df_options_filtered = df_options_filtered.sort_values(by="chance_of_profit_short", ascending=False)[
    [
        "symbol",
        # "id",
        "type",
        # "min_ticks",
    ]
    + numeric_columns
    # + ["high_fill_rate_price_diff"]
]

# df_options_filtered.loc[df_options_filtered.high_fill_rate_buy_price <= df_options_filtered.high_fill_rate_sell_price]
df_options_filtered

Min buy price: $0.20
Max buy price: $1.40


,symbol,type,chance_of_profit_short,strike_price,ask_size,bid_size,last_trade_size,ask_price,bid_price,last_trade_price
29,SPY,put,0.99,410.0,566,268,1.0,0.24,0.23,0.23
37,SPY,put,0.98,455.0,442,504,245.0,0.42,0.40,0.42


In [7]:
option_to_purchase = None

if len(df_options_filtered) > 0:
    option_to_purchase = df_options_filtered.to_dict(orient="records")[0]
    print("Option to purchase:")
    print(json.dumps(option_to_purchase, indent=4))
else:
    print("No option to purchase")

Option to purchase:
{
    "symbol": "SPY",
    "type": "put",
    "chance_of_profit_short": 0.99,
    "strike_price": 410.0,
    "ask_size": 566,
    "bid_size": 268,
    "last_trade_size": 1.0,
    "ask_price": 0.24,
    "bid_price": 0.23,
    "last_trade_price": 0.23
}


In [8]:
# print("Canceling all option orders...")
# task.robinhood_api.cancel_all_option_orders()

In [9]:
# if option_to_purchase is not None:
#     print(
#         json.dumps(
#             task.robinhood_api.order_buy_option_limit(
#                 positionEffect="open",
#                 creditOrDebit="debit",
#                 timeInForce="gtc",
#                 quantity=1,
#                 symbol=task.symbol,
#                 expirationDate=expiration_date,
#                 optionType=option_to_purchase["type"],
#                 strike=option_to_purchase["strike_price"],
#                 price=option_to_purchase["ask_price"],
#             ),
#             indent=4,
#         )
#     )

In [10]:
# if option_to_purchase is not None:
#     print(
#         json.dumps(
#             task.robinhood_api.order_sell_option_limit(
#                 positionEffect="close",
#                 creditOrDebit="credit",
#                 timeInForce="gtc",
#                 quantity=1,
#                 symbol=task.symbol,
#                 expirationDate=expiration_date,
#                 optionType=option_to_purchase["type"],
#                 strike=option_to_purchase["strike_price"],
#                 price=round(option_to_purchase["ask_price"] * 1.1, 2),
#             ),
#             indent=4,
#         )
#     )